In [78]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
library(tidyverse); 
library(lubridate); 
library(scales);
library(magrittr); 
library(dplyr);
})

In [79]:
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

In [3]:
options(repr.plot.width = 15, repr.plot.height = 10)

# Introduction

__April 20, 2022__ 

[Parent QA ticket](https://phabricator.wikimedia.org/T305442) | [QA ticket](https://phabricator.wikimedia.org/T306558)

# Instrumentation note
The related events will be stored in schema desktopwebuiactionstracking with event.name = 'ui.toc'.
Sample rate:
20% for pilot wikis



# QA DesktopWebUIActionsTracking schema

## Check event.action, skin, skin version, anon users, edit count bucket

In [33]:
query <- 
"
SELECT event.action, event.skin, event.skinVersion, event.isAnon,
event.editCountBucket, COUNT(1) AS clicks, 
COUNT(DISTINCT event.token) AS session_tokens
FROM event.desktopwebuiactionstracking 
WHERE year = 2022 AND month = 4 
AND event.name = 'ui.toc'
GROUP BY event.action,event.skin, event.skinVersion, event.isAnon,
event.editCountBucket
"

In [34]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [35]:
df

action,skin,skinversion,isanon,editcountbucket,clicks,session_tokens
<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>
click,vector-2022,2,false,5-99 edits,63507,21390
click,vector-2022,2,false,100-999 edits,52877,12496
click,vector-2022,2,true,0 edits,2483914,1821429
click,vector-2022,2,false,1-4 edits,23480,9435
click,vector-2022,2,false,1000+ edits,108750,21633
click,vector-2022,2,false,0 edits,26142,11445


## By wikis

In [98]:
query <- 
"
SELECT wiki,  COUNT(1) AS clicks, 
COUNT(DISTINCT event.token) AS session_tokens, min(meta.dt) as first_event_ts
FROM event.desktopwebuiactionstracking 
WHERE year = 2022 AND month IN (3, 4) 
AND event.name = 'ui.toc'
GROUP BY wiki
ORDER BY session_tokens DESC, first_event_ts
LIMIT 10000
"

In [99]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



__Note:__ 

Events are enabled on 135 wikis.

frwiki has most clicks on toc events and sessions, 5.9 times of the 2nd place ptwiki. Need to check the sample rate. 

## By date

In [42]:
query <- 
"
SELECT to_date(meta.dt) AS date_time, COUNT(1) AS clicks, 
COUNT(DISTINCT event.token) AS session_tokens
FROM event.desktopwebuiactionstracking 
WHERE year = 2022 
AND event.name = 'ui.toc'
GROUP BY to_date(meta.dt)
"

In [43]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [44]:
df

date_time,clicks,session_tokens
<chr>,<int>,<int>
2022-03-29,400,270
2022-03-30,7103,4894
2022-03-31,140134,103428
2022-04-01,161531,117455
2022-04-02,136477,97035
2022-04-03,157052,111906
2022-04-04,180706,133069
2022-04-05,176664,130172
2022-04-06,177019,130644


__Note:__

Events are available since March 29, 2022. 

## By agent type

In [106]:
query<- "

SELECT  COUNT(1) AS clicks, 
COUNT(DISTINCT event.token) AS session_tokens
FROM event.desktopwebuiactionstracking 
WHERE year = 2022 AND month = 4 
AND event.name = 'ui.toc' AND user_agent_map['device_family']='Spider'
"

In [107]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [108]:
df

clicks,session_tokens
<int>,<int>
6,5


## bot user

In [103]:
query <- 
"
SELECT useragent.is_bot, COUNT(1) AS clicks, 
COUNT(DISTINCT event.token) AS session_tokens
FROM event.desktopwebuiactionstracking 
WHERE year = 2022 AND month = 4 
AND event.name = 'ui.toc'
GROUP BY useragent.is_bot
"

In [104]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [105]:
df

is_bot,clicks,session_tokens
<chr>,<int>,<int>
false,3098412,2127775
true,7,6


## Screen resolution

In [61]:
query <- 
"
SELECT DISTINCT event.viewportSizeBucket
FROM event.desktopwebuiactionstracking 
WHERE year = 2022 AND month = 4 AND day=17
AND event.name = 'ui.toc'
"

In [62]:
df <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [63]:
df

viewportsizebucket
<chr>
NULL
>2000px
720-999
720px-999px
<320px
1000-1199
1000px-1199px
1200-2000
1200px-2000px


In [71]:
query <- "
select distinct wiki
from event.desktopwebuiactionstracking 
where year = 2022 and month = 4 
and event.name = 'ui.toc'
AND event.viewportSizeBucket='1200-2000'
"

In [72]:
df_1 <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [73]:
df_1

wiki
<chr>
brwiki
frwiktionary
itwikisource
specieswiki
kawiki
metawiki
frwikiquote
zhwikiquote
arywiki


In [74]:
query <- "
select distinct wiki
from event.desktopwebuiactionstracking 
where year = 2022 and month = 4 
and event.name = 'ui.toc'
AND event.viewportSizeBucket='1200px-2000px'
"

In [75]:
df_2<- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [76]:
df_2

wiki
<chr>
frwiktionary
itwikisource
specieswiki
frwikinews
kawiki
metawiki
frwikiquote
nahwiki
arywiki


__Note:__

Check the different formats of screen sizes by wiki. 

In the `desktopwebuiactionstracking` schema , the `viewportsizebucket` field has mixed formats for the screen size. For example, for the same screen size, it was recored as 1200-2000 or 1200px-2000px. It shows on all wikis and all days since the instrument was enabled.
